# TP4 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP3.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [27]:
import sqlite3

In [28]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [29]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBB:
  0: NB(INT)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
achat:
  0: NV(INT) not null *1
  1: NB(INT) not null *2
  2: DATES(DATE)
  3: LIEU(TEXT)
  4: QTE(INT)
buveurs:
  0: NB(INT) not null *1
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
geo:
  0: LIEU(TEXT) not null *1
  1: REGION(TEXT)
producteur:
  0: NP(INT) not null *1
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
qte_prod:
  0: NP(INT) not null *2
  1: NV(INT) not null *1
  2: QTE(INT)
quantity:
  0: NP(INT) not null *2
  1: NV(INT) not null *1
  2: QTE(INT)
vins:
  0: NV(INT) not null *1
  1: CRU(TEXT)
  2: DEGRE(FLOAT)
  3: MILL(INT)
vins_cons:
  0: NV(N

Again, we will use **`%%sql`** magic for our queries

In [30]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP3

In [31]:
%%sql
DROP TABLE IF EXISTS geo;
CREATE TABLE 'geo'(
    'LIEU'        TEXT   NOT NULL,
    'REGION'      TEXT,
    PRIMARY KEY ('LIEU')
);
INSERT INTO 'geo' (LIEU,REGION) 
   SELECT DISTINCT LIEU,REGION
   FROM Master2
   WHERE LIEU IS NOT NULL;


 * sqlite:///wine.db
Done.
Done.
18 rows affected.


[]

In [32]:
%%sql
DROP TABLE IF EXISTS producteur;
CREATE TABLE 'producteur'(
   'NP'           INT    NOT NULL,
   'NOM'           TEXT,  
   'PRENOM'        TEXT,
   'REGION'        TEXT,
   PRIMARY KEY ('NP'),
   FOREIGN KEY ('REGION')
        REFERENCES geo ('REGION')
);
INSERT INTO 'producteur' (NP, NOM, PRENOM,REGION) 
   SELECT DISTINCT NP, NOM, PRENOM,REGION
   FROM Master1
   WHERE NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.
124 rows affected.


[]

In [33]:
%%sql
DROP TABLE IF EXISTS vins;
CREATE TABLE 'vins'(
   'NV'           INT    NOT NULL,
   'CRU'           TEXT,  
   'DEGRE'        FLOAT,
   'MILL'        INT,
   PRIMARY KEY ('NV')
);
INSERT INTO 'vins' (NV,CRU,DEGRE,MILL) 
   SELECT DISTINCT NV,CRU,DEGRE,MILL
   FROM Master1
   WHERE NV IS NOT NULL;


 * sqlite:///wine.db
Done.
Done.
102 rows affected.


[]

In [34]:
%%sql
DROP TABLE IF EXISTS qte_prod;
CREATE TABLE 'qte_prod'(
   'NP'           INT   NOT NULL,
   'NV'           INT   NOT NULL,  
   'QTE'          INT,
   PRIMARY KEY ('NV','NP'),
   FOREIGN KEY ('NV')
        REFERENCES vins ('NV'),
    FOREIGN KEY ('NP')
        REFERENCES producteur ('NP')
);
INSERT INTO 'qte_prod' (NP,NV,QTE) 
   SELECT DISTINCT NP,NV,QTE
   FROM Master1
   WHERE NV IS NOT NULL AND NP IS NOT NULL;


 * sqlite:///wine.db
Done.
Done.
140 rows affected.


[]

In [35]:
%%sql
DROP TABLE IF EXISTS buveurs;
CREATE TABLE 'buveurs'(
    'NB'        INT   NOT NULL,
    'NOM'       TEXT,  
    'PRENOM'    TEXT,
    'TYPE'      TEXT,
    PRIMARY KEY ('NB')
);
INSERT INTO 'buveurs' (NB,NOM,PRENOM,TYPE) 
   SELECT DISTINCT NB,NOM,PRENOM,TYPE
   FROM Master2
   WHERE NB IS NOT NULL;


 * sqlite:///wine.db
Done.
Done.
100 rows affected.


[]

In [36]:
%%sql
DROP TABLE IF EXISTS achat;
CREATE TABLE 'achat'(
    'NV'        INT     NOT NULL,
    'NB'        INT     NOT NULL,
    'DATES'     DATE,
    'LIEU'      TEXT,
    'QTE'       INT,
    PRIMARY KEY ('NV','NB'),
    FOREIGN KEY ('LIEU')
        REFERENCES geo ('LIEU'),
    FOREIGN KEY ('NV')
        REFERENCES vins ('NV'),
    FOREIGN KEY ('NB')
        REFERENCES buveurs ('NB')
);
INSERT INTO 'achat' (NV,NB,DATES,LIEU,QTE) 
   SELECT DISTINCT NV,NB,DATES,LIEU,QTE
   FROM Master2
   WHERE NV IS NOT NULL AND NB IS NOT NULL ;

 * sqlite:///wine.db
Done.
Done.
73 rows affected.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [37]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS 
SELECT * 
FROM buveurs b
WHERE b.TYPE='gros' OR b.TYPE='moyen'

 * sqlite:///wine.db
Done.
Done.


[]

In [38]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [39]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS 
        SELECT b.NB,NOM,PRENOM,TYPE
        FROM buveurs b
        LEFT OUTER JOIN achat a on b.NB = a.NB
        WHERE a.NB IS NULL AND a.NV IS NULL

 * sqlite:///wine.db
Done.
Done.


[]

In [40]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [41]:
%%sql
DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS 
        SELECT * 
            FROM buveurs 
    EXCEPT
        SELECT b.NB,NOM,PRENOM,TYPE
            FROM buveurs b
            LEFT OUTER JOIN achat a on b.NB = a.NB
            WHERE a.NB IS NULL AND a.NV IS NULL

 * sqlite:///wine.db
Done.
Done.


[]

In [42]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [43]:
%%sql
DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
    SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE
        FROM achat a
    LEFT OUTER JOIN vins v ON a.NV =v.NV
        WHERE strftime('%Y',DATES)="1983"
        GROUP BY LIEU,CRU;

 * sqlite:///wine.db
Done.
Done.


[]

In [44]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

Answer: As QTE is not part of the new aggregated table, there would be no point in ordering with ascended order over it

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [45]:
%%sql 
DROP TABLE IF EXISTS RBB;
CREATE TABLE RBB AS
SELECT * FROM bons_buveurs ORDER BY NB ASC;

 * sqlite:///wine.db
Done.
Done.


[]

In [46]:
# Test
%sql SELECT * FROM RBB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

In [47]:
%%sql
SELECT bb.NB,NOM,PRENOM,TYPE,SUM(QTE) AS total 
FROM buveurs bb
LEFT OUTER JOIN achat a ON bb.NB = a.NB
WHERE lower(TYPE)!='gros'
GROUP BY bb.NB
HAVING SUM(QTE)>=100


 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,total
2,Artaud,Antonin,moyen,583
5,Audiberti,Jacques,petit,113
9,Ajar,Emile,petit,140
44,Gide,Andre,petit,171


Find the instances to update (schema may differ from the one in your table)

Update instances

In [54]:
%%sql
SELECT bb.NB,NOM,PRENOM,TYPE,SUM(QTE) AS total 
FROM buveurs bb
LEFT OUTER JOIN achat a ON bb.NB = a.NB
WHERE lower(TYPE)!='gros'
GROUP BY bb.NB
HAVING SUM(QTE)>100

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,total
2,Artaud,Antonin,moyen,583
5,Audiberti,Jacques,petit,113
9,Ajar,Emile,petit,140
44,Gide,Andre,petit,171


In [78]:
%%sql
UPDATE buveurs
SET TYPE = 'gros'
WHERE buveurs.NB=(
SELECT bb.NB
FROM buveurs bb
LEFT OUTER JOIN achat a ON bb.NB = a.NB
WHERE lower(TYPE)!='gros'
GROUP BY bb.NB
HAVING SUM(QTE)>100);

 * sqlite:///wine.db
1 rows affected.


[]

In [80]:
%%sql
SELECT bb.NB
FROM buveurs bb
LEFT OUTER JOIN achat a ON bb.NB = a.NB
WHERE lower(TYPE)!='gros'
GROUP BY bb.NB
HAVING SUM(QTE)>100

 * sqlite:///wine.db
Done.


NB
5
9
44


#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

Answer: 

In [ ]:
%%sql
SELECT * FROM RBB 
EXCEPT 
SELECT * FROM bons_buveurs

#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [61]:
%%sql 
DROP TABLE IF EXISTS RBA;
CREATE TABLE RBA AS
SELECT * FROM buveurs_asec ORDER BY NB ASC;

 * sqlite:///wine.db
Done.
Done.


[]

In [62]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [67]:
%%sql
INSERT INTO RBA (NB,NOM,PRENOM,TYPE )
VALUES (101,'Gardette','Mathieu','petit');

 * sqlite:///wine.db
1 rows affected.


[]

In [68]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [70]:
%%sql
SELECT * FROM RBA 
EXCEPT 
SELECT * FROM buveurs_asec

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
101,Gardette,Mathieu,petit


Answer:

Here I'am, at the end of the table. As expected since I've added myself.

___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

Answer:

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.

Answer: